In [2]:
import sys
!{sys.executable} -m pip install fpdf -q
!{sys.executable} -m pip install pandas -q
!{sys.executable} -m pip install plotly-express -q
!{sys.executable} -m pip install kaleido -q



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd 
import plotly.express as px 
from fpdf import FPDF  

In [15]:
# Define the plotly template. Some other examples:
# "plotly", "ggplot2", "seaborn", "simple_white", "plotly_dark", "plotly_white", ..
plotly_template = "presentation"

In [16]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / r"C:\Users\Eyosi\Downloads\sales (1).db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

In [17]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [18]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [19]:
# Print the DataFrame
print(df)

      sale_date  total_sales
0    2022-01-01         1095
1    2022-01-02          555
2    2022-01-03          150
3    2022-01-04          975
4    2022-01-06          395
..          ...          ...
335  2022-12-26         1315
336  2022-12-27          685
337  2022-12-28         1545
338  2022-12-29          740
339  2022-12-31          850

[340 rows x 2 columns]


In [20]:
# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    340 non-null    object
 1   total_sales  340 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [21]:
# Convert sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    340 non-null    datetime64[ns]
 1   total_sales  340 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.4 KB


In [22]:
# Set the sale_date column as the index
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2022-01-01,1095
2022-01-02,555
2022-01-03,150


In [23]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample('M').sum()
df_monthly

,total_sales
sale_date,
2022-01-31,23600
2022-02-28,22400
2022-03-31,29660
2022-04-30,19225
2022-05-31,30015
2022-06-30,24395
2022-07-31,27525
2022-08-31,20835
2022-09-30,19285


In [24]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,23600,Jan
2022-02-28,22400,Feb
2022-03-31,29660,Mar
2022-04-30,19225,Apr
2022-05-31,30015,May
2022-06-30,24395,Jun
2022-07-31,27525,Jul
2022-08-31,20835,Aug
2022-09-30,19285,Sep


In [25]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

In [26]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)
df

,product_name,total_sales
0,Product A,58850
1,Product B,25350
2,Product C,84600
3,Product D,42320
4,Product E,68760


In [27]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

In [28]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)
df

,customer_name,total_sales
0,Grace Lee,33720
1,Bob Smith,31285
2,Alice Jones,29000
3,Jane Doe,28460
4,Emily Davis,27855
5,Frank Wilson,27355
6,Isabel Garcia,26235
7,Henry Chen,25920
8,David Brown,25045
9,John Doe,25005


In [29]:
# Create the Plotly figure
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

# Show the plot
fig.show()

# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

In [30]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output folder
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10)  # Add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

# Save the PDF document to a file on disk
pdf.output(output_dir / "sales_report.pdf", "F")

''

In [31]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales,customer_segment
0,8,Grace Lee,33720,High Value
1,3,Bob Smith,31285,High Value
2,4,Alice Jones,29000,Medium Value
3,2,Jane Doe,28460,Medium Value
4,6,Emily Davis,27855,Medium Value
5,7,Frank Wilson,27355,Medium Value
6,10,Isabel Garcia,26235,Medium Value
7,9,Henry Chen,25920,Low Value
8,5,David Brown,25045,Low Value
9,1,John Doe,25005,Low Value


In [32]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,1,John Doe,25005
1,2,Jane Doe,28460
2,3,Bob Smith,31285
3,4,Alice Jones,29000
4,5,David Brown,25045
5,6,Emily Davis,27855
6,7,Frank Wilson,27355
7,8,Grace Lee,33720
8,9,Henry Chen,25920
9,10,Isabel Garcia,26235


In [33]:
# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df

,customer_id,customer_name,total_sales,customer_segment
7,8,Grace Lee,33720,High Value
2,3,Bob Smith,31285,High Value
3,4,Alice Jones,29000,Medium Value
1,2,Jane Doe,28460,Medium Value
5,6,Emily Davis,27855,Medium Value
6,7,Frank Wilson,27355,Medium Value
9,10,Isabel Garcia,26235,Medium Value
8,9,Henry Chen,25920,Low Value
4,5,David Brown,25045,Low Value
0,1,John Doe,25005,Low Value


In [34]:
# Close the connection
conn.close()